In [ ]:
from ultralytics import YOLO
import cv2
import time
from datetime import datetime, timedelta
import pytz
from kafka import KafkaProducer
import json

# Initialize Kafka Producer
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Load YOLO model
# model = YOLO(r"D:\YOLODATA\yolo\runs\train\yolov8_testlog_continue_20250414_184408\weights\best.pt")
# model = YOLO(r"C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250428_1507512\weights\best.pt")
# model =YOLO(r"C:\Users\jj111\Desktop\new_train\runs\train\yolov8_testlog_20250430_1637332\weights\best.pt")
model = YOLO(r"C:\Users\jj111\Desktop\new_train\kafka_yolo_model.pt")
# Open the camera
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1088)

# Define parking spots
PARKING_SPOTS = [
    {"id": 1, "box": (950, 620, 1000, 670)},
    {"id": 2, "box": (1210, 640, 1260, 690)},
    {"id": 3, "box": (1450, 570, 1500, 620)}
]

# Check if center point is inside the bounding box (with margin)
def is_center_in_box(cx, cy, box, margin=25):
    x1, y1, x2, y2 = box
    return (x1 - margin) <= cx <= (x2 + margin) and (y1 - margin) <= cy <= (y2 + margin)

# Initialize time tracking for message sending
last_sent_time = datetime.now()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame")
        break

    results = model.predict(frame, conf=0.3, stream=True, imgsz=(1088, 1920), verbose=False)

    for result in results:
        boxes = result.boxes
        class_ids = boxes.cls.cpu().numpy()
        coords = boxes.xyxy.cpu().numpy()

        # Filter only class_id = 2 (Car)
        car_boxes = [coords[i] for i, cls_id in enumerate(class_ids) if int(cls_id) == 2]

        statuses = {spot["id"]: "Empty" for spot in PARKING_SPOTS}
        for x1, y1, x2, y2 in car_boxes:
            cx = (x1 + x2) / 2
            cy = (y1 + y2) / 2
            for spot in PARKING_SPOTS:
                if is_center_in_box(cx, cy, spot["box"]):
                    statuses[spot["id"]] = "Occupied"

        # Send message only every 5 seconds
        now = datetime.now()
        if (now - last_sent_time).total_seconds() >= 5:
            la_time = now.astimezone(pytz.timezone('America/Los_Angeles')).isoformat()
            message = {
                "timestamp": la_time,
                "spots": [{"id": spot_id, "status": status} for spot_id, status in statuses.items()]
            }
            producer.send('parking-spot-status', message)
            print(f"[Kafka Sent] {message}")
            last_sent_time = now

        # Display frame
        annotated = result.plot()
        for spot in PARKING_SPOTS:
            x1, y1, x2, y2 = spot["box"]
            color = (0, 255, 0) if statuses[spot["id"]] == "Empty" else (0, 0, 255)
            label = f"Spot {spot['id']}: {statuses[spot['id']]}"
            cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
            cv2.putText(annotated, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        cv2.imshow("YOLO Parking Monitor", annotated)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[Kafka Sent] {'timestamp': '2025-05-07T13:17:05.786129-07:00', 'spots': [{'id': 1, 'status': 'Empty'}, {'id': 2, 'status': 'Empty'}, {'id': 3, 'status': 'Empty'}]}
[Kafka Sent] {'timestamp': '2025-05-07T13:17:10.811007-07:00', 'spots': [{'id': 1, 'status': 'Empty'}, {'id': 2, 'status': 'Empty'}, {'id': 3, 'status': 'Empty'}]}
[Kafka Sent] {'timestamp': '2025-05-07T13:17:15.824832-07:00', 'spots': [{'id': 1, 'status': 'Empty'}, {'id': 2, 'status': 'Empty'}, {'id': 3, 'status': 'Empty'}]}
[Kafka Sent] {'timestamp': '2025-05-07T13:17:20.844184-07:00', 'spots': [{'id': 1, 'status': 'Empty'}, {'id': 2, 'status': 'Empty'}, {'id': 3, 'status': 'Empty'}]}
[Kafka Sent] {'timestamp': '2025-05-07T13:17:25.849151-07:00', 'spots': [{'id': 1, 'status': 'Empty'}, {'id': 2, 'status': 'Empty'}, {'id': 3, 'status': 'Empty'}]}
[Kafka Sent] {'timestamp': '2025-05-07T13:17:30.870751-07:00', 'spots': [{'id': 1, 'status': 'Empty'}, {'id': 2, 'status': 'Empty'}, {'id': 3, 'status': 'Empty'}]}
[Kafka Sent] {'t

KeyboardInterrupt: 

: 